In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def boolean_fuction(x):
    y = []
    for i in x:
        x1,x2,x3,x4,x5,x6,x7,x8,x9,x10=i
        y.append((x1^x2) & (x3 | x4) ^ (x5 & x6) | (x7 ^ x8) & (x9 ^ x10))
    return y

In [4]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def sidmoid_der(x):
    s = sigmoid(x)
    return s*(1-s)

In [5]:
def tanh(x):
    # ex = np.exp(x)
    # enx = np.exp(-1)
    # return (ex-enx)/(ex+enx)
    return np.tanh(x)
def tanh_der(x):
    return 1-tanh(x)**2

In [6]:
def relu(x):
    return np.maximum(0,x)
def relu_der(x):
    return (x>0).astype(float)

In [7]:
X = np.array([[int(b) for b in format(i,"010b")] for i in range(0,2**10)])

In [8]:
y = np.array(boolean_fuction(X)).reshape(-1,1)

In [9]:
y.shape, X.shape

((1024, 1), (1024, 10))

In [10]:
def initilizing(x,h1,h2,o):
    np.random.seed(42)
    W1 = np.random.randn(x,h1) * 0.1
    b1 = np.zeros((1,h1))
    W2 = np.random.randn(h1,h2) * 0.1
    b2 = np.zeros((1,h2))
    W3 = np.random.randn(h2,o) * 0.1
    b3 = np.zeros((1,o))
    return W1, b1, W2, b2, W3, b3
# def initialize_parameters(input_size, hidden1, hidden2, output_size):
#     np.random.seed(42)
#     W1 = np.random.randn(input_size, hidden1) * 0.1
#     b1 = np.zeros((1, hidden1))
#     W2 = np.random.randn(hidden1, hidden2) * 0.1
#     b2 = np.zeros((1, hidden2))
#     W3 = np.random.randn(hidden2, output_size) * 0.1
#     b3 = np.zeros((1, output_size))
#     return W1, b1, W2, b2, W3, b3

In [11]:
# def forward(x,w1,b1,w2,b2,w3,b3):
#     z1 = np.dot(x,w1)+b1
#     a1 = relu(z1)

#     z2 = np.dot(z1,w2) + b2
#     a2 = tanh(z2)

#     z3 = np.dot(z2,w3) + b3
#     a3 = sigmoid(z3)

#     return z1,a1,z2,a2,z3,a3
def forward(X, W1, b1, W2, b2, W3, b3):
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = tanh(z2)
    z3 = np.dot(a2, W3) + b3
    a3 = sigmoid(z3)
    return z1, a1, z2, a2, z3, a3

In [12]:
# def backward(X, y, z1, a1, z2, a2, z3, a3, W2, W3):
#     m = X.shape[0]
#     dz3 = a3 - y
#     dW3 = np.dot(a2.T, dz3) / m
#     db3 = np.sum(dz3, axis=0, keepdims=True) / m

#     dz2 = np.dot(dz3, W3.T) * tanh_der(z2)
#     dW2 = np.dot(a1.T, dz2) / m
#     db2 = np.sum(dz2, axis=0, keepdims=True) / m

#     dz1 = np.dot(dz2, W2.T) * relu_der(z1)
#     dW1 = np.dot(X.T, dz1) / m
#     db1 = np.sum(dz1, axis=0, keepdims=True) / m

#     return dW1, db1, dW2, db2, dW3, db3
def backward(X, y, z1, a1, z2, a2, z3, a3, W2, W3):
    m = X.shape[0]
    dz3 = a3 - y
    dW3 = np.dot(a2.T, dz3) / m
    db3 = np.sum(dz3, axis=0, keepdims=True) / m
    dz2 = np.dot(dz3, W3.T) * tanh_der(z2)
    dW2 = np.dot(a1.T, dz2) / m
    db2 = np.sum(dz2, axis=0, keepdims=True) / m
    dz1 = np.dot(dz2, W2.T) * relu_der(z1)
    dW1 = np.dot(X.T, dz1) / m
    db1 = np.sum(dz1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2, dW3, db3

In [13]:
def train(X, y, epochs=1000, lr=0.1):
    W1, b1, W2, b2, W3, b3 = initilizing(10, 16,8, 1)
    for epoch in range(epochs):
        z1, a1, z2, a2, z3, a3 = forward(X, W1, b1, W2, b2, W3, b3)
        
        loss = -np.mean(y * np.log(a3 + 1e-8) + (1 - y) * np.log(1 - a3 + 1e-8))

        dW1, db1, dW2, db2, dW3, db3 = backward(X, y, z1, a1, z2, a2, z3, a3, W2, W3)

        W1 -= lr * dW1
        b1 -= lr * db1
        W2 -= lr * dW2
        b2 -= lr * db2
        W3 -= lr * dW3
        b3 -= lr * db3

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

    return W1, b1, W2, b2, W3, b3


In [14]:
W1, b1, W2, b2, W3, b3 = train(X, y, epochs=1000, lr=0.5)
_, _, _, _, _, predictions = forward(X, W1, b1, W2, b2, W3, b3)
predictions_binary = (predictions > 0.5).astype(int)
accuracy = np.mean(predictions_binary == y)
print(f"Final Accuracy: {accuracy*100:.2f}%")

Epoch 0, Loss: 0.6943
Epoch 100, Loss: 0.6803
Epoch 200, Loss: 0.6731
Epoch 300, Loss: 0.6226
Epoch 400, Loss: 0.5020
Epoch 500, Loss: 0.4021
Epoch 600, Loss: 0.2426
Epoch 700, Loss: 0.1383
Epoch 800, Loss: 0.0802
Epoch 900, Loss: 0.0424
Final Accuracy: 100.00%


In [18]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8
    return -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))


In [15]:
np.random.permutation(5)

array([3, 1, 0, 2, 4])

In [19]:
def train_sgd(X, y, epochs=100, lr=0.1):
    W1, b1, W2, b2, W3, b3 = initilizing(10, 5,5, 1)
    m = X.shape[0]

    for epoch in range(epochs):
        total_loss = 0
        indices = np.random.permutation(m)
        X_shuffled = X[indices]
        y_shuffled = y[indices]

        for i in range(m):
            xi = X_shuffled[i].reshape(1, -1)
            yi = y_shuffled[i].reshape(1, -1) 

            z1, a1, z2, a2, z3, a3 = forward(xi, W1, b1, W2, b2, W3, b3)
            loss = binary_cross_entropy(yi, a3)
            total_loss += loss

            dW1, db1, dW2, db2, dW3, db3 = backward(xi, yi, z1, a1, z2, a2, z3, a3, W2, W3)
            W1 -= lr * dW1
            b1 -= lr * db1
            W2 -= lr * dW2
            b2 -= lr * db2
            W3 -= lr * dW3
            b3 -= lr * db3

        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {total_loss/m:.4f}")

    return W1, b1, W2, b2, W3, b3


In [22]:
W1, b1, W2, b2, W3, b3 = train_sgd(X, y, epochs=200, lr=0.1)
_, _, _, _, _, predictions = forward(X, W1, b1, W2, b2, W3, b3)
predictions_binary = (predictions > 0.5).astype(int)
accuracy = np.mean(predictions_binary == y)
print(f"Final Accuracy: {accuracy*100:.2f}%")

Epoch 1, Loss: 0.6869
Epoch 11, Loss: 0.6043
Epoch 21, Loss: 0.4804
Epoch 31, Loss: 0.4923
Epoch 41, Loss: 0.4751
Epoch 51, Loss: 0.4899
Epoch 61, Loss: 0.4547
Epoch 71, Loss: 0.4566
Epoch 81, Loss: 0.4704
Epoch 91, Loss: 0.4502
Epoch 101, Loss: 0.4511
Epoch 111, Loss: 0.4805
Epoch 121, Loss: 0.4538
Epoch 131, Loss: 0.4538
Epoch 141, Loss: 0.4606
Epoch 151, Loss: 0.4553
Epoch 161, Loss: 0.4539
Epoch 171, Loss: 0.4753
Epoch 181, Loss: 0.4678
Epoch 191, Loss: 0.4509
Final Accuracy: 83.11%


In [196]:
def adam_update(params, grads, v, s, t, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
    updated_params = []
    for i in range(len(params)):
        
        v[i] = beta1 * v[i] + (1 - beta1) * grads[i]
 
        s[i] = beta2 * s[i] + (1 - beta2) * (grads[i] ** 2)
        
        v_corrected = v[i] / (1 - beta1 ** t)
        s_corrected = s[i] / (1 - beta2 ** t)

        param = params[i] - lr * v_corrected / (np.sqrt(s_corrected) + epsilon)
        updated_params.append(param)
    return updated_params, v, s

In [197]:
def train_adam(X, y, epochs=1000, lr=0.01):
    W1, b1, W2, b2, W3, b3 = initialize_parameters(10, 16, 8, 1)
    params = [W1, b1, W2, b2, W3, b3]
    v = [np.zeros_like(p) for p in params]
    s = [np.zeros_like(p) for p in params]

    for epoch in range(1, epochs + 1):
        z1, a1, z2, a2, z3, a3 = forward(X, W1, b1, W2, b2, W3, b3)
        loss = binary_cross_entropy(y, a3)

        dW1, db1, dW2, db2, dW3, db3 = backward(X, y, z1, a1, z2, a2, z3, a3, W2, W3)
        grads = [dW1, db1, dW2, db2, dW3, db3]

        params, v, s = adam_update(params, grads, v, s, epoch, lr)

        W1, b1, W2, b2, W3, b3 = params

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.5f}")

    return W1, b1, W2, b2, W3, b3

In [202]:
W1, b1, W2, b2, W3, b3 = train_adam(X, y)
_, _, _, _, _, predictions = forward(X, W1, b1, W2, b2, W3, b3)
predictions_binary = (predictions > 0.5).astype(int)
accuracy = np.mean(predictions_binary == y)
print(f"Final Accuracy: {accuracy*100:.2f}%")

Epoch 100, Loss: 0.23923
Epoch 200, Loss: 0.03263
Epoch 300, Loss: 0.01000
Epoch 400, Loss: 0.00523
Epoch 500, Loss: 0.00334
Epoch 600, Loss: 0.00237
Epoch 700, Loss: 0.00179
Epoch 800, Loss: 0.00141
Epoch 900, Loss: 0.00115
Epoch 1000, Loss: 0.00095
Final Accuracy: 100.00%


array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])